# 데이터 적재 및 모델 구축에 필요한 함수 정의, 모듈 호출

In [1]:
import os
import datetime as dt
import math
import copy
import re
import random
import warnings

import pandas as pd
import numpy as np

#import tensorflow as tf
#from catboost import CatBoostRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.base import clone
#import konlpy

warnings.filterwarnings(action='ignore')

In [2]:
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [3]:
def drop_col_feat_imp(model, X_train, y_train, random_state=42):
    '''
    구동 원리:
    1. 모든 변수 (feature) 를 피처로 사용했을 때의 R2-Score 계산
    2. 변수 한 개씩을 제외하며 R2-Score 기준으로 성능 확인
    3. ((전체 변수 이용했을 때의 R2-Score) - (변수 1개를 제외했을 때의 R2-Score)) 값 계산
    4. 3번 값이 음수라면 해당 변수를 제거하는 편이 좋다고 판단, 양수라면 해당 변수가 포함되어 있는 편이 좋다고 결론
    '''

    model_clone = clone(model)
    model_clone.random_state = random_state
    model_clone.fit(X_train, y_train)
    benchmark_score = model_clone.score(X_train, y_train)

    importances = []
    n_iter = 1
    for col in X_train.columns:
        print('iteration: {}'.format(n_iter))
        model_clone = clone(model)
        model_clone.random_state = random_state
        model_clone.fit(X_train.drop(col, axis=1), y_train)
        drop_col_score = model_clone.score(X_train.drop(col, axis=1), y_train)
        importances.append(benchmark_score - drop_col_score)
        n_iter += 1
        clear_output()

    dic = {'feature': X_train.columns, 'importances': importances}
    importances_df = pd.DataFrame(dic)

    return importances_df

In [4]:
def Jaccard_similarity(doc1, doc2):
    doc1 = set(doc1)
    doc2 = set(doc2)
    return len(doc1 & doc2) / len(doc1 | doc2)

In [5]:
def intersection_similarity(doc1, doc2):
    '''
    2개의 상품명을 비교하여 공통적으로 존재하는 단어 수 반환 (교집합 기반) 
    '''
    doc1 = set(doc1)
    doc2 = set(doc2)
    return len(doc1 & doc2)

In [6]:
def inter_info_ratio(doc1,doc2):
    '''
    (두 개의 상품명 doc1, doc2 을 비교하여 공통적으로 존재하는 단어 수 / 하나의 상품명 (doc1) 의 단어 수) 계산
    -> '공통 정보 비율' 값을 도출
    '''
    return intersection_similarity(doc1,doc2)/len(set(doc1))

In [7]:
# 상품의 유사도를 기반으로 과거 실적을 매핑하기 위해 정의한 클래스

class MapPastPred():

    def __init__(self, df2019, df2020):

        self.df2019_items = np.unique(df2019.상품코드.values)
        self.df2019_mother = np.unique(df2019.마더코드.values)
        self.df2019 = df2019
        self.df2020 = df2020

    def mapping(self, option='name'): # 2019년도에 판매된 상품 - 2020년도 6월에 판매된 상품을 유사한 것들끼리 상품코드 기준으로 매핑
        df = pd.DataFrame()
        상품2020 = []
        상품2019 = []
        iir=[]
        noise = []
        item2019 = self.df2019[['상품코드', '상품명', '마더코드', '판매단가', '상품군']]
        item2019 = item2019.drop_duplicates('상품코드')
        elimination = '[(초특가)(\(.+\))(무이자)(세트)(1+1)(일시불)(\[.+\])(\d+종)(\d+팩)(\+)(\-)]'
        item2019['상품명'] = [re.sub(elimination, '', i)
                           for i in item2019.상품명]

        season = np.unique(self.df2019[(
            self.df2019.방송일시 >= '2019-05-01') & (self.df2019.방송일시 < '2019-09-01')].상품코드.values)
        season_cat = ['의류', '가전', '농수축']

        tag = konlpy.tag.Kkma()
        item2019['토큰'] = [tag.nouns(i) for i in item2019.상품명]

        for i in np.unique(self.df2020.상품코드.values):
            if i in self.df2019_items:
                상품2020.append(i)
                상품2019.append(i)
                name_token = item2019[item2019.상품코드 == i].토큰.values[0]
                iir.append(len(name_token))
            else:
                상품2020.append(i)

                similarity = pd.DataFrame()
                mother = self.df2020[self.df2020.상품코드 == i].마더코드.values[0]
                cat = self.df2020[self.df2020.상품코드 == i].상품군.values[0]
                price = self.df2020[self.df2020.상품코드 == i].판매단가.values[0]

                if mother in self.df2019_mother:
                    temp = item2019[item2019.마더코드 == mother][[
                        '상품코드', '상품명', '토큰', '판매단가']]

                    if len(temp) == 0:
                        temp = item2019[item2019.상품군 == cat][[
                            '상품코드', '상품명', '토큰', '판매단가']]
                        if self.df2020[self.df2020.상품코드 == i].상품군.values[0] in season_cat:
                            temp = temp[temp.상품코드.isin(season)]

                else:
                    temp = item2019[item2019.상품군 == cat][[
                        '상품코드', '상품명', '토큰', '판매단가']]
                    if self.df2020[self.df2020.상품코드 == i].상품군.values[0] in season_cat:
                        temp = temp[temp.상품코드.isin(season)]

                temp_sub = temp.copy()
                temp = temp[(temp.판매단가 > price*0.7) & (temp.판매단가 < price*1.3)]

                if len(temp) == 0:
                    temp = temp_sub

                temp.drop_duplicates('상품명', keep='first', inplace=True)

                if option == 'name':
                    code = []
                    similar = []

                    name = self.df2020[self.df2020.상품코드 == i].상품명.values[0]
                    name = re.sub(elimination, '', name)
                    token = tag.nouns(name)

                    for k in range(len(temp)):
                        code.append(temp.상품코드.values[k])
                        similar.append(intersection_similarity(token, temp.토큰.values[k]))

                    similarity['상품코드'] = code
                    similarity['유사도'] = similar
                    similarity = similarity.sort_values(
                        '유사도', ascending=False).reset_index(drop=True)
                    iir.append(max(similar))
                    noise.append(1)

                    if len(similarity) > 1:

                        if len(similarity[similarity.유사도 == max(similarity.유사도.values)]) > 1:
                            temp2 = similarity[similarity.유사도 == max(
                                similarity.유사도.values)]
                            temp2 = temp2.merge(
                                temp[['상품코드', '판매단가']], on='상품코드', how='left')
                            temp2['가격차이'] = (
                                temp['판매단가'] - self.df2020[self.df2020.상품코드 == i].판매단가.values[0]).apply(np.abs)
                            temp2 = temp2.sort_values(
                                '가격차이').reset_index(drop=True)
                            상품2019.append(temp2.상품코드.values[0])

                        else:
                            상품2019.append(similarity.상품코드.values[0])

                    else:
                        상품2019.append(similarity.상품코드.values[0])

                elif option == 'onlyprice':
                    temp['판매단가'] = (temp.판매단가 - price).apply(np.abs)
                    temp = temp.sort_values('판매단가')
                    상품2019.append(temp.상품코드.values[0])

        df['pred상품코드'] = 상품2020
        df['past상품코드'] = 상품2019
        df['공통정보량'] = iir

        self.mapping_table = df
        return df

    def mapping_transform(self, option='name'): # 상품코드 기준으로 과거실적 매핑
        mapped_table = self.mapping(option=option)
        df2020 = self.df2020.copy()
        df2019 = self.df2019.copy()

        past_performance = df2019.pivot_table(index='상품코드', values='취급액',
                                              aggfunc='mean')
        past_performance.reset_index(inplace=True)
        past_performance.columns = ['상품코드', '과거실적']

        transformed_table = pd.merge(mapped_table, past_performance, left_on='past상품코드',
                                     right_on='상품코드', how='left')
        transformed_table = transformed_table[['pred상품코드','공통정보량', '과거실적']]

        df2020 = pd.merge(df2020, transformed_table, left_on='상품코드', right_on='pred상품코드',
                          how='left')

        del df2020['pred상품코드']
        
        
        for i in df2020.loc[df2020.공통정보량==0].index:
            cate = df2020.loc[i,'상품군']
            price = df2020.loc[i,'판매단가']
            df2020.loc[i,'과거실적'] = df2019[(df2019.상품군 == cate)&\
                                          (df2019.판매단가>price*0.7)&\
                                          (df2019.판매단가<price*1.3)].취급액.mean()
            
        self.mapping_transform_table = df2020
        return df2020
    
def mean_performance(df): # 상품 코드 별 취급액 평균 계산
    past_performance = df.pivot_table(index='상품코드', values='취급액', aggfunc='mean')
    past_performance.reset_index(inplace=True)
    past_performance.columns = ['상품코드', '과거실적']
    df1 = pd.merge(df, past_performance, on='상품코드', how='left')
    return df1
    
def noise_making(df2019,ratio=0.3):
    '''
    과거 판매 이력이 없는 상품이 많은 상황을 가정하여 몇 개 상품의 실적을 의도적으로 가리고,
    매핑 다른 상품코드의 과거실적을 대신 매핑하는 방식으로 학습 데이터 재구축
    '''
    df = df2019.copy()
    item = list(np.unique(df.상품코드))
    random.shuffle(item)
    idx = int(len(item)*ratio)
    noise_item = item[:idx]
    normal_item = item[idx:]
    
    normal = df[df.상품코드.isin(normal_item)]
    noise = df[df.상품코드.isin(noise_item)]
    
    mapp = MapPastPred(normal,noise)
    table = mapp.mapping_transform(option='name')
    
    df['공통정보량'] = 1
    df = mean_performance(df)
    
    noise_table = pd.concat([df,table])
    noise_table.reset_index(inplace=True, drop=True)
    
    return {'noise_table':noise_table,'only_normal':df,'only_noise':table}

# StackingModel을 생성하는 Class 선언 및 예측, 저장


## StackingModel

In [8]:
tf.random.set_seed(7777)
np.random.seed(7777)
random.seed(7777)
class StackingRegressor():
    def __init__(self,
                 model1=CatBoostRegressor(verbose=False,
                                               loss_function='MAE',
                                               random_state=7777,
                                               bagging_temperature=3,
                                               depth=8,
                                               learning_rate=0.1),
                 model2=tf.keras.models.Sequential([
                     tf.keras.layers.Flatten(),
                     tf.keras.layers.Dense(180, activation='relu'),
                     tf.keras.layers.Dense(360, activation='relu'),
                     tf.keras.layers.Dropout(0.3,seed=7777),
                     tf.keras.layers.Dense(720, activation='relu'),
                     tf.keras.layers.Dense(180, activation='relu'),
                     tf.keras.layers.Dense(90, activation='relu'),
                     tf.keras.layers.Dense(1)
                 ])):
        
        self.model1 = model1
        self.model2 = model2
        model2.compile(optimizer='adam', loss='mape', metrics=['mae', 'mse'])

    def fit(self, train_set, target_col='취급액'):

        self.target_col = target_col
        self.train = train_set

        feature_col = [
            '상품군', '판매단가', '월', '요일', '시', '분', '주말', '공휴일', '지불방식', '노출분',
            '누적노출', '마감시간여부', '계절_numeric', '일평균기온', '일최저기온', '일최고기온', '일별강수량',
            '태풍발생', '일별상대습도', '일평균풍속', '홈쇼핑검색량', 'NS검색량', '클릭비율', '과거실적'
        ]

        self.cat_feature = [
            '과거실적', '시', '판매단가', '마감시간여부', '누적노출', '분', '상품군', '클릭비율', '요일',
            '홈쇼핑검색량', '공휴일', '일평균풍속', '주말', '계절_numeric', 'NS검색량', '태풍발생',
            '노출분', '일평균기온', '일별강수량', '월', '일별상대습도', '일최저기온', '지불방식', '일최고기온'
        ]

        self.feature_not_col = [
            '방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '취급액', '년', '일', '방송날짜',
            '계절_categoric'
        ]

        cate = np.unique(self.train.상품군.values)
        self.replace_ = {cate[i]: i for i in range(len(cate))}

        train = mean_performance(self.train)

        X_train = train[self.cat_feature]
        y_train = train[self.target_col]
        X_train.상품군 = X_train.상품군.replace(self.replace_)

        self.model1.fit(X_train, y_train)
        self.cat_pred = self.model1.predict(X_train)
        
        dnn_train = self.train.copy()
        categorical = ['상품군','월', '요일', '시', '분', '주말', '지불방식', '계절_numeric', '태풍발생']
        one_hot = pd.get_dummies(dnn_train, columns=categorical)
        train = mean_performance(one_hot)
        
        feature_not_col = [
            '방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '취급액', '년', '일', '방송날짜',
            '계절_categoric'
        ]
        
        X_train = train.drop(feature_not_col, axis=1)
        y_train = train['취급액']
        
        X_train['cat_pred'] = self.cat_pred
        self.feature_ = X_train.columns
        
        
        self.dnn_train = X_train

        with tf.device("/device:CPU:0"):
            self.model2.fit(X_train.values,
                        y_train.values,
                        epochs=10,
                        batch_size=30)

    def predict(self, test_set):

        self.test = test_set

        mapping_ = MapPastPred(self.train, self.test)
        cat_val = mapping_.mapping_transform()

        X_test = cat_val[self.cat_feature]
        self.y_test = cat_val[self.target_col]

        X_test.상품군 = X_test.상품군.replace(self.replace_)
        X_test = X_test[self.cat_feature]

        self.cat_y_pred = self.model1.predict(X_test)
        
        #colums 수 맞추는 작업
        dnn_train = self.train.copy()
        dnn_val = self.test.copy()

        dnn_train['구분'] = 'train'
        dnn_val['구분'] = 'val'

        all_table = pd.concat([dnn_train, dnn_val])
        categorical = ['월', '요일', '시', '분', '주말', '지불방식', '계절_numeric', '태풍발생']
        one_hot = pd.get_dummies(all_table, columns=categorical)

        dnn_train = one_hot[one_hot.구분 == 'train']
        dnn_val = one_hot[one_hot.구분 == 'val']

        mapp = MapPastPred(dnn_train, dnn_val)
        val = mapp.mapping_transform()

        del val['공통정보량']

        train = mean_performance(dnn_train)

        feature_not_col = [
            '방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '취급액', '년', '일', '방송날짜',
            '계절_categoric'
        ]

        dt = pd.concat([train, val])
        dt = pd.get_dummies(dt, columns=['상품군'])
        dnn_train = dt[dt.구분 == 'train']
        dnn_val = dt[dt.구분 == 'val']

        del dnn_val['구분']
        

        X_test = dnn_val.drop(feature_not_col, axis=1)
        
        X_test['cat_pred'] = self.cat_y_pred
        X_test=X_test[self.feature_]
        
        self.dnn_test = X_test
        
        with tf.device("/device:CPU:0"):
            self.predict_ = self.model2.predict(X_test.values)

        self.score = MAPE(self.y_test, self.predict_.reshape(-1))

        return self.predict_

In [2]:
train,val = pd.read_excel('../data/internal/preprocessed_data/train_data/실적_전처리.xlsx'),pd.read_excel('../data/internal/preprocessed_data/test_data/평가_전처리.xlsx')

In [ ]:
# 위의 데이터 임포트 코드 실행 순서가 2인 것에 대한 설명: 본 코드 파일을 제출용 폴더에 옮긴 후에도 데이터 적재가 잘 되는지 마지막으로 확인

In [10]:
stack_ = StackingRegressor()

In [11]:
stack_.fit(train)

Train on 35379 samples
Epoch 1/10
35379/35379 [==============================] - 5s 128us/sample - loss: 23.6718 - mae: 4293732.0000 - mse: 53948295675904.0000
Epoch 2/10
35379/35379 [==============================] - 3s 87us/sample - loss: 22.4323 - mae: 4068617.7500 - mse: 48482790408192.0000
Epoch 3/10
35379/35379 [==============================] - 3s 91us/sample - loss: 22.3016 - mae: 4021020.5000 - mse: 47138520170496.0000
Epoch 4/10
35379/35379 [==============================] - 3s 93us/sample - loss: 22.1578 - mae: 3990498.0000 - mse: 47261589438464.0000
Epoch 5/10
35379/35379 [==============================] - 4s 99us/sample - loss: 22.1802 - mae: 4009853.7500 - mse: 47597913899008.0000
Epoch 6/10
35379/35379 [==============================] - 3s 89us/sample - loss: 22.2021 - mae: 4006333.7500 - mse: 47405672169472.0000
Epoch 7/10
35379/35379 [==============================] - 3s 89us/sample - loss: 22.1447 - mae: 4000678.0000 - mse: 47054697005056.0000
Epoch 8/10
35379/35379 [

In [ ]:
y_pred = stack_.predict(val)
val['취급액'] = y_pred.reshape(-1)

In [ ]:
val.to_excel('NS_shop_예측결과.xlsx',index=False)

In [ ]:
val.취급액

In [17]:
val.취급액

0         810204.25
1        2302592.75
2        3262947.25
3       16053125.00
4       27957666.00
           ...     
2711    27309784.00
2712    26311884.00
2713    27387650.00
2714     9236733.00
2715    12256555.00
Name: 취급액, Length: 2716, dtype: float32